In [1]:
from Data import get_min_descriptionsNorm_triples_relations, LOGGER_FILES, PKLS_FILES
from utils.utils import read_cached_array, cache_array, get_logger
from transformers import BertTokenizerFast
from flashtext import KeywordProcessor


c:\Users\antoi\Desktop\uni\NLP_WIKI\code\BRASK\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:



k = 10
DESCRIPTION_MAX_LENGTH = 128
descs, triples, relations, aliases = get_min_descriptionsNorm_triples_relations(k)
golden_triples_file = PKLS_FILES["golden_triples"][k]
len(descs)

54

In [3]:
def get_entity_start_end_idxs(entity_description_tokens, entity_to_find_aliases, tokenizer):
    """
        I try to find using flashtext the entity to find in the description, if found return the idxs 
    """ 
    spans = []
    alias_token_lists = []
    for alias in entity_to_find_aliases:
        toks = tokenizer.tokenize(alias)
        alias_token_lists.append((toks, alias))
    
    L = len(entity_description_tokens)
    for toks, alias in alias_token_lists:
        m = len(toks)
        if m == 0 or m > L:
            continue
        for i in range(0, L - m + 1):
            if entity_description_tokens[i : i + m] == toks:
                spans.append((i, i + m - 1, alias))
    return spans

In [4]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

buid_logger = get_logger("build_golden_triples", LOGGER_FILES["build_golden_triples"])

sentences_ids = list(descs.keys())
sentences = list(descs.values())
encoded = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt", max_length=DESCRIPTION_MAX_LENGTH)
sentences_tokens = [tokenizer.convert_ids_to_tokens(sen) for sen in encoded["input_ids"]]

golden_triples = []
buid_logger.info(f"Building golden triples for {len(descs)}: \n We will log every description sentence and then the aliases for the head, tail of the description triples and log the extracted golden triples.. By triples I mean double of (head, tail)")


for sen_tokens, sen_id in zip(sentences_tokens, sentences_ids):
    buid_logger.info(f"\n\nSentence : {descs[sen_id]}")
    buid_logger.info(f"Sentence tokens: {sen_tokens}")
    sen_trps = []
    trpls = triples[sen_id]
    triples_found_logs = []
    for h, r, t in trpls:
        buid_logger.info(f"\n\t next triple: ")
        h_aliases = aliases[h]
        buid_logger.info(f"\thead aliases: {h_aliases}")
        trp_h = get_entity_start_end_idxs(sen_tokens, h_aliases, tokenizer)
        if len(trp_h) == 0:
            buid_logger.info(f"\t \t no head aliases matched with the sentence")
            continue
        buid_logger.info(f"\t \thead aliases matched {trp_h}")
        
        t_aliases = aliases[t]
        buid_logger.info(f"\ttail aliases: {t_aliases}")
        trp_t = get_entity_start_end_idxs(sen_tokens, t_aliases, tokenizer)
        
        if len(trp_t) == 0:
            buid_logger.info(f"\t \tno tail aliases matched with the sentence")
            continue 
        buid_logger.info(f"\t \ttail aliases matched {trp_t}")
        
        relation_aliases = relations[r]
        for matched_head in trp_h:
            for matched_tail in trp_t:
                sen_trps.append(( (matched_head[0], matched_head[1]) ,  (matched_tail[0], matched_tail[1])  ))
                triples_found_logs.append(f"\n\n\t******** TRIPLE FOUND:  (HEAD, TAIL): ({matched_head[2]}, {matched_tail[2]}), (head_idxs, tail_idxs): ({(matched_head[0], matched_head[1]) ,  (matched_tail[0], matched_tail[1])}) \n\tRelation aliases: {relation_aliases}"   )
                
    for l in triples_found_logs:
        buid_logger.info(l)
               
    golden_triples.append(sen_trps)



cache_array(golden_triples, golden_triples_file)
        

Array chached in file ./data/dictionaries/golden_triples/golden_triples_min_10.pkl


In [10]:
# pip install flashtext
from flashtext import KeywordProcessor

tail_aliases = ['sexy ann', 'Sexy Ann']

sentence = 'System in Blue" is a song by German music group Systems in Blue, released as promo single in 2005. it was produced by Rolf Kohler, Detlef Wiedeke and Thomas Widrat.it was re-released in 2009 by Akasa Records, containing an extended version produced by Talking System.'
# 1) build your keyword‐processor
kp = KeywordProcessor(case_sensitive=False, )
for alias in tail_aliases:
    kp.add_keyword(alias)

# 2) extract all matches with their char spans
matches = kp.extract_keywords(sentence, span_info=True)
# matches is a list of tuples: [(keyword, start_char, end_char), …]

matches

[]

In [ ]:
for d_id, d_txt in descs.items():
    triples = triples[d_id]
    for h, _, t in triples:
        
    break 